In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Read data from file
with open('data.txt', 'r') as f:
    data = f.readlines()

# Define player data
players = []
for i in range(0, len(data), 6):
    player = {
        'defense_ranks': np.array([int(j) for j in data[i].split(',')], dtype=np.float32),
        'defense_encodings': np.array([int(j) for j in data[i+1].split(',')], dtype=np.float32),
        'fantasy_scores': np.array([float(j) for j in data[i+2].split(',')], dtype=np.float32),
        'skill_score': np.array([float(j) for j in data[i+3].split(',')], dtype=np.float32),
        'weeks_encodings': np.array([int(j) for j in data[i+4].split(',')], dtype=np.float32),
        'season_encodings': np.array([int(j) for j in data[i+5].split(',')], dtype=np.float32)
    }
    players.append(player)

# Process inputs and targets for each player
X_train, Y_train, X_test, Y_test = [], [], [], []
 
for player in players[:40]:
    player_inputs = torch.from_numpy(player['defense_ranks']).view(1, 50, 1).float()
    player_inputs = torch.cat((player_inputs, torch.from_numpy(player['defense_encodings']).view(1, 50, 1).float()), dim=2)
    player_inputs = torch.cat((player_inputs, torch.from_numpy(player['skill_score']).view(1, 50, 1).float()), dim=2)
    player_inputs = torch.cat((player_inputs, torch.from_numpy(player['weeks_encodings']).view(1, 50, 1).float()), dim=2)
    player_inputs = torch.cat((player_inputs, torch.from_numpy(player['season_encodings']).view(1, 50, 1).float()), dim=2)
    X_train.append(player_inputs)
    Y_train.append(torch.from_numpy(player['fantasy_scores']).view(1, 50, 1).float())
  
for player in players[40:41]:
    player_test_inputs = torch.from_numpy(player['defense_ranks']).view(1, 50, 1).float()
    player_test_inputs = torch.cat((player_test_inputs, torch.from_numpy(player['defense_encodings']).view(1, 50, 1).float()), dim=2)
    player_test_inputs = torch.cat((player_test_inputs, torch.from_numpy(player['skill_score']).view(1, 50, 1).float()), dim=2)
    player_test_inputs = torch.cat((player_test_inputs, torch.from_numpy(player['weeks_encodings']).view(1, 50, 1).float()), dim=2)
    player_test_inputs = torch.cat((player_test_inputs, torch.from_numpy(player['season_encodings']).view(1, 50, 1).float()), dim=2)
    X_test.append(player_test_inputs)  
    Y_test.append(torch.from_numpy(player['fantasy_scores']).view(1, 50, 1).float())

# Combine inputs and targets for all players
X_train = torch.cat(X_train, dim=0)
Y_train = torch.cat(Y_train, dim=0)
X_test = torch.cat(X_test, dim=0)
Y_test = torch.cat(Y_test, dim=0)
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

(torch.Size([40, 50, 5]),
 torch.Size([40, 50, 1]),
 torch.Size([1, 50, 5]),
 torch.Size([1, 50, 1]))

In [113]:
# Define the neural network architecture
class RegressionModel(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, hidden_size)
        self.fc5 = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.fc3(out)
        out = self.fc4(out)
        out = self.fc5(out)
        return out

# Set hyperparameters
initial_learning_rate = 0.0003
reduced_learning_rate = 0.0003
increased_learning_rate = 0.0005

# Create the model
model = RegressionModel(5, 1000)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=initial_learning_rate)

total = 0
params = list(model.parameters())
for i in params:
    total += len(i)
total

8002

In [115]:
# Training loop for each player
batch_size = 29
num_players = len(X_train)
num_batches = num_players // batch_size  # Only full batches
epoch_runs = 200
reduced_lr_epoch = False
increased_lr_epoch = False

# Training loop
for epoch_run in range(epoch_runs):
    for batch in range(num_batches):
        # Forward pass
        outputs = model(X_train[batch * batch_size: (batch + 1) * batch_size])
        loss = criterion(outputs, Y_train[batch * batch_size: (batch + 1) * batch_size])

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Manually reduce the learning rate at the specified epoch
        if reduced_lr_epoch == True:
            for param_group in optimizer.param_groups:
                param_group['lr'] = reduced_learning_rate

        if increased_lr_epoch == True:
            for param_group in optimizer.param_groups:
                param_group['lr'] = increased_learning_rate
        
        # Print the loss for every epoch
        if epoch_run + 1 == epoch_runs:
            print(f"Batch {batch+1}/{num_batches}, Epoch {epoch_run+1}/{epoch_runs}, Loss: {loss.item()}")
    
# Make predictions for the player's test inputs
with torch.no_grad():
    player_predicted = model(X_test)

print() # blank line
# Print the predictions and original scores for the player
for i in range(len(X_test[0])):
    for j in range(len(X_test)):
        print(f"Player {j+1}: Skill Score: {round(X_test[j][i][2].item(), 2)}, Defense Rank: {X_test[j][i][0].item()} " + 
              f"Predicted Fantasy Score: {round(player_predicted[j][i].item(), 2)}, Actual Fantasy Score: {round(Y_test[j][i].item(), 2)}")
    print() # blank line

Batch 1/1, Epoch 200/200, Loss: 47.720462799072266

Player 1: Skill Score: 7.17, Defense Rank: 0.0 Predicted Fantasy Score: 0.22, Actual Fantasy Score: 0.0

Player 1: Skill Score: 7.17, Defense Rank: 0.0 Predicted Fantasy Score: 0.25, Actual Fantasy Score: 0.0

Player 1: Skill Score: 7.17, Defense Rank: 0.0 Predicted Fantasy Score: 0.28, Actual Fantasy Score: 0.0

Player 1: Skill Score: 7.17, Defense Rank: 0.0 Predicted Fantasy Score: 0.3, Actual Fantasy Score: 0.0

Player 1: Skill Score: 7.17, Defense Rank: 0.0 Predicted Fantasy Score: 0.3, Actual Fantasy Score: 0.0

Player 1: Skill Score: 7.17, Defense Rank: 0.0 Predicted Fantasy Score: 0.29, Actual Fantasy Score: 0.0

Player 1: Skill Score: 7.17, Defense Rank: 24.0 Predicted Fantasy Score: 6.79, Actual Fantasy Score: 3.7

Player 1: Skill Score: 7.17, Defense Rank: 8.0 Predicted Fantasy Score: 6.9, Actual Fantasy Score: 1.1

Player 1: Skill Score: 7.17, Defense Rank: 1.0 Predicted Fantasy Score: 5.47, Actual Fantasy Score: 8.3

Playe